In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} for training")
device

Using cuda for training


'cuda'

In [3]:
import multiprocessing; print(multiprocessing.cpu_count()) # 打印 CPU 核心数

24


In [4]:
#train_dataloader=DataLoader(training_data, batch_size=64)
#test_dataloader=DataLoader(test_data, batch_size=64)
train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True, num_workers=4, pin_memory=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)

这两行代码是用 PyTorch 的 DataLoader 来封装 FashionMNIST 数据集，用于 高效加载数据。相比于原始的 DataLoader(training_data, batch_size=64)，新代码引入了 shuffle=True、num_workers=4 和 pin_memory=True，这些参数可以 提高数据加载的速度和训练效率。

DataLoader 负责：
批量加载数据（Batching）
随机打乱数据（Shuffling）
并行加载数据（num_workers）
优化 GPU 训练数据传输（pin_memory）


batch_size=64	批量大小	每次加载 64 张图片，减少显存占用，提高 GPU 利用率；
shuffle=True	随机打乱	训练集打乱数据，避免模型过拟合到数据顺序，提高泛化能力；
shuffle=False	不打乱	测试集不打乱数据，确保评估指标稳定；
num_workers=4	多线程数据加载	使用 4 个 CPU 线程并行加载数据，加速训练（通常等于CPU核心数的一半）；
pin_memory=True	固定内存	预加载到 固定 CPU 内存，加速 GPU 数据传输，提高性能, GPU训练时推荐开启。

In [5]:
class NeuralNetwork(nn.Module):#define NeutralNnetwork
    def __init__(self):
        super().__init__()
        self.flatten=nn.Flatten()
        self.linear_relu_stack=nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self,x):
        x=self.flatten(x)
        logits=self.linear_relu_stack(x)
        return logits
    
model = NeuralNetwork()
model.to(device)#把model搬到GPU

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [6]:
learning_rate=1e-3
batch_size=64#这行可删，dataloader里面的batchsize才算数
epochs=5

In [7]:
loss_fn=nn.CrossEntropyLoss()

In [8]:
optimizer=torch.optim.SGD(model.parameters(), lr=learning_rate)

In [9]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size=len(dataloader.dataset) # 训练集总样本数
    model.train()
# Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices

    
    for batch, (X,y)in enumerate(dataloader):#遍历数据加载器，取出 batch_size 大小的数据
        X, y = X.to(device), y.to(device)  #  让数据移动到 GPU
        # Compute prediction and loss
        pred=model(X)#前向传播，得到predicted output
        loss = loss_fn(pred, y)#计算损失

        # Backpropagation
        loss.backward()#计算梯度
        optimizer.step()#利用优化器更新参数
        optimizer.zero_grad()#清空梯度，防止梯度累积

        #每100个batch输出一次损失
        if batch%100 == 0:
            loss, current =loss.item(), batch*batch_size+len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            

In [10]:
from tqdm import tqdm#加个进度条
def test_loop(dataloader,model,loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size=len(dataloader.dataset)
    num_batches=len(dataloader)
    test_loss, correct=0,0#初始化损失和正确分类计数

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():#关闭梯度计算
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)  #  让数据移动到 GPU
            pred=model(X)#前向传播
            test_loss+=loss_fn(pred,y).item()#累计损失
            correct+=(pred.argmax(1)==y).type(torch.float).sum().item()
            '''pred.argmax(1): 获取每个样本预测的类别（取最大概率的索引）
(pred.argmax(1) == y).type(torch.float).sum().item(): 计算 预测正确的样本数 并累加到 correct'''

    test_loss/=num_batches
    correct/=size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
     

In [11]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.295863  [   64/60000]
loss: 2.288517  [ 6464/60000]
loss: 2.266764  [12864/60000]
loss: 2.257055  [19264/60000]
loss: 2.236307  [25664/60000]
loss: 2.222915  [32064/60000]
loss: 2.222806  [38464/60000]
loss: 2.197294  [44864/60000]
loss: 2.175478  [51264/60000]
loss: 2.135356  [57664/60000]
Test Error: 
 Accuracy: 35.1%, Avg loss: 2.148313 

Epoch 2
-------------------------------
loss: 2.147570  [   64/60000]
loss: 2.164401  [ 6464/60000]
loss: 2.122908  [12864/60000]
loss: 2.063793  [19264/60000]
loss: 2.014570  [25664/60000]
loss: 2.022108  [32064/60000]
loss: 1.984013  [38464/60000]
loss: 2.021985  [44864/60000]
loss: 1.931021  [51264/60000]
loss: 1.905525  [57664/60000]
Test Error: 
 Accuracy: 58.1%, Avg loss: 1.884229 

Epoch 3
-------------------------------
loss: 1.949434  [   64/60000]
loss: 1.793685  [ 6464/60000]
loss: 1.757614  [12864/60000]
loss: 1.783540  [19264/60000]
loss: 1.736668  [25664/60000]
loss: 1.730875  [32064/600